# Source Visibility with agilepy

The AGILE satellite operated from April 23, 2007 to January 18, 2024 observing thousands of astrophysical sources.

The AGILE spacecraft operated in "pointing mode" from the beginning of the mission to October 15, 2009, completing 101 observation blocks (OBs).
The OBs usually consisted of predefined long exposures, drifting about 1 deg per day with respect to the initial boresight direction to obey solar panels constraints.

In November 2009, the attitude control system was reconfigured, and scientific operations were performed "spinning mode" until the end of the mission.
AGILE scanned ≈ 80% sky daily (exposure of ≈ 7 · 106 cm2 s) with an angular velocity of about 0.8 deg s−1, performing 200 passes per day on the same sky region.

The visibility of a source thus depend on the off-axis angle with respect to the satellite pointing direction.

The purpose of this notebook is to show how to use the class `AGVisibility` to determine the AGILE pointing direction and the offset of a source of interest.
The class requires only the "log files", which are the AGILE spacecraft files, to perform its task.
If a Fermi Spacecraft file is provided, the same operation can be performed on the file for comparison purposes.


In [ ]:
# Import the relevant class
from agilepy.api.AGVisibility import AGVisibility
from agilepy.utils.AstroUtils import AstroUtils

from astropy.coordinates import SkyCoord
from astropy.time import Time

## Setup

This tutorial uses the available test dataset on blazar 3C 454.3, from 2010-11-13 to 2010-11-21 (MJD 55513-55521).

In [ ]:
# Prepare the YAML Configuration File.
# You can create it manually or use AGVisibility.getConfiguration()

outputDir    = "/home/flareadvocate/workspace/shared_dir/"
confFilePath = "/home/flareadvocate/workspace/shared_dir/tutorial_visibility.yaml"
logFile      = "$AGILE/agilepy-test-data/test_dataset_agn/LOG/LOG.index"
fermi_SCFile = "$AGILE/agilepy-test-data/visibility/Fermi_test_SC00.fits"

AGVisibility.getConfiguration(
    confFilePath=confFilePath,
    # Output
    outputDir=outputDir,
    userName="my_name",
    sourceName="vis-source",
    verboselvl=0,
    # Input
    logFile=logFile,
    fermiSpacecraftFile=fermi_SCFile,
    # Selection
    step=1,
    timeType="tt",
    tmin="null",
    tmax="null",
    # Source
    coord1="null",
    coord2="null",
    frame="icrs",
)

The `YAML` file has the following sections and field:
- `output`: common to all `agilepy` classes, sets the following output fields: `outdir`, `filenameprefix`, `sourcename`, `username`, `verboselvl`
- `input`: input spacecraft files.
    - `logFile`: path to the index file listing relevant AGILE log files to be extracted.
    - `fermiSpacecraftFile`: the path to the Fermi spacecraft file. Optional.
- `selection`: arguments that define the timing of the information extracted.
    - `step`: time interval in seconds between two consecutive points in the resulting table. Minimum accepted value: 0.1.
    - `timeType`: the time format of `tmin` and `tmax`. AGILE seconds can be expressed as `tt`.
    - `tmin`: inferior observation time limit to analize. If not set, default value is TT = 104371200 (2007-04-23 00:00:00)
    - `tmax`: superior observation time limit to analize. If not set, default value is TT = 632620800 (2024-01-18 00:00:00)
- `source`: optional, include arguments to define the coordinates of the source used to compute the offset. The coordinates can also be set or changed when using the class. They are defined by:
    - `frame`: reference frame for `astropy.coordinates.SkyCoord`
    - `coord1`, `coord2`: source coordinates in degrees.


In [ ]:
# Create the AGVisibility object
ag_vis = AGVisibility(confFilePath)

# Print Options
ag_vis.printOptions()

In [ ]:
# You can use the setOptions() method to change the configuration:

# Set coordinates of 3C 454.3
ag_vis.setOptions(coord1=343.496, coord2=16.151, frame='icrs')

# Set time range appropriate for the test dataset
ag_vis.setOptions(tmin=AstroUtils.convert_time_to_agile_seconds(Time(55513, format='mjd')),
                  tmax=AstroUtils.convert_time_to_agile_seconds(Time(55521, format='mjd')),
                  timetype="TT"
                  )

# Check configuration
ag_vis.printOptions()

Note on time conversion: the `agilepy.utils.AstroUtils.AstroUtils` class offers utility functions for the conversion between AGILE seconds and other data formats, handled through the `astropy.time.Time` class.

In [ ]:
# Convert from AGILE seconds to astropy Time
t_example = AstroUtils.convert_time_from_agile_seconds(104371200)
display(f"TT=104371200 corresponds to MJD {t_example.mjd}, ISO {t_example.iso}, UNIX {t_example.unix}")

# Convert from astropy Time to AGILE seconds
t_example = AstroUtils.convert_time_to_agile_seconds(Time(54213, format='mjd'))
display(f"MJD=54213.0 corresponds to TT {t_example}")


## AGILE Visibility
The AGILE pointing coordinates can be computed with `AGVisibility.computePointingDirection()`.
The arguments of the function are taken by default from the configuration, or they can be overriden if provided explicitly as arguments of the method.

The AGILE data is extracted from the columns `TIME`, `ATTITUDE_RA_Y`, `ATTITUDE_DEC_Y` of the log files and stored in an `astropy.table.Table`.

The output columns provide the start and stop times of the bin analysed in TT and MJD, the AGILE pointing coordinates (Right Ascension and Declination) and the mean offset of the source in degrees in the bin.
The distance between two consecutive bins is the `step` parameter.

The table is saved by default in the agilepy output directory as a `.csv` file.

In [ ]:
agile_visibility_table = ag_vis.computePointingDirection(writeFiles=True)

# Show
display(agile_visibility_table)

# Note: The table can also be accessed as a property of AGVisibility:
# ag_vis.agileVisibility

## Fermi Visibility

The Fermi pointing coordinates can be computed with `AGVisibility.getFermiPointing()`.
The arguments of the function are taken by default from the configuration, or they can be overriden if provided explicitly as arguments of the method.

The Fermi data is extracted from the columns `START`, `STOP`, `RA_SCZ`, `DEC_SCZ` of the spacecraft file and stored in an `astropy.table.Table`.

The output columns provide the start and stop times of the bin analysed in TT, MJD, Fermi MET seconds, the Fermi pointing coordinates (Right Ascension and Declination) and the mean offset of the source in degrees in the bin.

The table is saved by default in the agilepy output directory as a `.csv` file.

In [ ]:
fermi_visibility_table = ag_vis.getFermiPointing(writeFiles=True)

# Show the table, accessed as a property of AGVisibility:
display(ag_vis.fermiVisibility)

## Plot
`AGVisibility` offers an utility function to plot the time series of the Offaxis angle.
Parameters are:
- `sourceCoordinates` (`astropy.coordinates.SkyCoord`): the coordinates of the source used to compute the offaxis angle, If None, the offaxis angle must already be computed in the visibility table.
- `maxOffaxis` (`float`): maximum offaxis angle in degrees to be shown in the visibility plot.
- `mjd_limits` (`tuple(float,float)`, optional): Plot limits in MJD.
- `plotFermi` (`bool`): if `True`, plot the Fermi offaxis angle.
- `plotHistogram` (`bool`): if `True`, plot the histogram of the offaxis angles.
- `saveImage` (`bool`): if `True`, save the images in the output directory.
- `showPositionPanels` (`bool`): if `True`, show the RA and DEC time series of the satellite direction.

In [ ]:
# Plot
ag_vis.plotVisibility(mjd_limits=(55519, 55519.5), maxOffaxis=60,
                      plotFermi=True, plotHistogram=True, saveImages=True,
                      showPositionPanels=False
                      )